### 7. Bildbearbeitung 1

#### 7.1. Bilddatei lesen und speichern
<center>

![alt text](cpp1_a7_abb_7_1_dreifach.png)<br>
Abbildung 7.1.: Grauwertbild in der Datei `cpp1_a7_abb_7_1_dreifach.pgm`
</center>
In der Datei `cpp1_a7_abb_7_1_dreifach.pgm` ist ein Grauwertbild (Abbildung 7.1) der Größe 512 × 256 Bildpunkten
gespeichert. Der Dateianfang ist in Liste 7.1 dargestellt (siehe auch Tabelle 7.1). Eine Bildzeile (512
Grauwerte) ist dabei in 32 aufeinanderfolgenden Dateizeilen gespeichert.

<center>

Liste 7.1: Beginn der pgm-Datei `cpp1_a7_abb_7_1_dreifach.pgm`
![alt text](cpp1_a7_liste_7_1_dreifach.png)
</center>

Schreiben Sie ein Python-Skript mit einer Funktion in welcher die Datei `cpp1_a7_abb_7_1_dreifach.pgm` eingelesen wird. Die Bildinformationen (Zeile 1-3) sollen zur Kontrolle ausgegeben werden. Das Bild wird in eine Doppel-Liste (Liste aus Zeilen mit Liste aus Pixeln (Spalten)) (bild-)zeilenweise eingelesen (siehe Liste 7.2). Beim Einlesen ist zu überprüfen, ob die Codierung des Dateityps (hier P2) richtig ist und ob das Bild in die angegebenen Bild-Dimensionen passt. Im Fehlerfall ist das Programm mit einer entsprechenden Meldung abzubrechen.<br>
Anschließend ist das Bild im selben Dateiformat in eine Datei `dreifach.out.pgm` zu speichern. Hierbei soll überprüft werden, ob das Schreiben in die Ausgabedatei erfolgreich war. Im Fehlerfall ist das Programm mit einer entsprechenden Meldung abzubrechen.<br>

Wenn auf dem Rechner ein Standardprogramm für die Anzeige von Bildern installiert und konfiguriert ist, kann dieser zur Anzeige des Bildes von Python aus gestartet werden:
```
import subprocess
...
subprocess.call("open " + imageFileName, shell=True)
```
Verwenden dieses, um charakteristische Bildbereiche zur Kontrolle zu vergrößern.

In [19]:
import os
import sys
import subprocess    

################################################################################
# Lesen eines PGM Image-Files in ein Dictionary.
# fileName: Pfad zur PGM-Image-Datei
# return  : dict['magic']: Magic der PGM-Datei
#           dict['width']: Breite des Bildes
#           dict['height']: Höhe des Bildes
#           dict['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
################################################################################
def readPgmImageFile(fileName):
    
    if (not os.path.exists(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert nicht!")
    if (not os.path.isfile(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert, ist aber keine Datei!")
    
    image = dict()
    image['pixels']=[]
    itemCount=0
    rowIdx=0
    colIdx=0
    with open(fileName, 'r', encoding='ascii') as file:
            text = file.read()
    for item in text.split():
        if (itemCount==0):
            image['magic']=item
        elif (itemCount==1):
            image['width']=int(item)
        elif (itemCount==2):
            image['height']=int(item)
        elif (itemCount==3):
            image['maxgray']=int(item)
        else:
            # Zeilen-Liste ermitteln
            row = image['pixels'][rowIdx] if len(image['pixels']) > rowIdx else []
            
            # Zeilenumbruch
            if len(row) >= image['width']:
                rowIdx+=1
                colIdx=0
                row=[]
            # Pixelwert einfügen
            pixel = int(item)
            if (pixel < 0 or pixel > image['maxgray']):
                raise ValueError(f"Pixel[{rowIdx}][{colIdx}]Erwarte Pixel 0..{image['maxgray']}, ist aber {pixel}")
            
            row.append(int(item))
            colIdx+=1
            
            # Zeile anhängen, wenn neu
            if (len(row) == 1):
                image['pixels'].append(row)
        itemCount+=1
    return image

################################################################################
# Schreiben eines PGM Image Dictionaries in eine Datei.
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
# fileName: Pfad zur PGM-Image-Datei
################################################################################
def writePgmImageFile(image, fileName):
    try:
        with open(fileName, 'w', encoding='ascii') as file:
            file.write(f"{image['magic']}\n")
            file.write(f"{image['width']}{image['height']:4d}\n")
            file.write(f"{image['maxgray']}\n")
            for row in image['pixels']:
                for pixel in row:
                    file.write(f"{pixel:4d} ")
                file.write("\n")
    except PermissionError as e:
        raise OSError(f"Rechte zum Schreiben der Datei {fileName} fehlen")
    except IOError as e:
        raise OSError(f"Fehler bei Schreiben der Datei {fileName}")
    
###########################################################
# Testskript
###########################################################
try:
    infile="cpp1_a7_abb_7_1_dreifach.pgm"
    outfile="dreifach.out.pgm"

    image = readPgmImageFile(infile)
    writePgmImageFile(image, outfile)

    subprocess.call("open " + outfile, shell=True)
except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e

Rechte zum Schreiben der Datei dreifach.out.pgm fehlen